### TODO
* Explain: Eligibility traces
* Explain: lambda methods
* Explain: True online SARSA lambda
* Explain: Tree Backup lambda
* code: Run and compare (on mountain car). Also compare with forwards view


### DONE


### NOTES

# Eligibility Traces for Reinforcement Learning

## $\lambda$-return
### n-step truncated $\lambda$-return

## Eligibility Trace

## Forward and Backward view

## True Online Methods

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
